In [3]:
!ls
import pandas as pd

BuildCSVs.ipynb    ParsedCSVs	 README.md	  dict
Exploration.ipynb  Parser.ipynb  WNdb-3.0.tar.gz  parsers


In [9]:
def createIndexDF(filePath):
    with open(filePath) as f:
        Indexdump = f.readlines()
    indexLines = []
    for i in Indexdump:
        if i[0] == " " or i[0] == ".":
            continue
        else:
            indexLines.append(i.strip())
    for i in range(len(indexLines)):
        splitLine = indexLines[i].split()
        ls = []
        ls.extend(splitLine[0:4])
        p_cnt = int(ls[-1])
        ls.append(splitLine[4:4 + p_cnt])
        sense_cntIndex = 4 + p_cnt
        ls.append(splitLine[sense_cntIndex])
        tagsense_cntIndex = sense_cntIndex + 1
        ls.append(splitLine[tagsense_cntIndex])
        remainingStartIndex = tagsense_cntIndex + 1
        ls.append(splitLine[remainingStartIndex:])
        indexLines[i] = ls
    df = pd.DataFrame(indexLines, columns = ['lemma', 'pos', 'synset_cnt', 'p_cnt', 'ptr_symbol', 'sense_cnt', 'tagsense_cnt', 'synset_offset'])
    return df

In [5]:
def createDataDF(filePath, Isverb=False):
    with open(filePath) as f:
        Datadump = f.readlines()
    dataLines = []
    for i in Datadump:
        if i[0] == " " or i[0] == ".":
            continue
        else:
            dataLines.append(i.strip())
    for i in range(len(dataLines)):
        splitLine = dataLines[i].split()
        ls = []
        ls.extend(splitLine[:4])
        wordCount = int(ls[-1], 16)
        currIndex = 4
        ls_words_and_lexids = []
        for j in range(wordCount):
            ls_words_and_lexids.append([splitLine[currIndex],splitLine[currIndex+1]])
            currIndex += 2
        ls.append(ls_words_and_lexids)
        p_cnt = splitLine[currIndex]
        ls.append(p_cnt)
        currIndex += 1
        ptrList = []
        for j in range(int(p_cnt)):
            ptrList.append([splitLine[currIndex],splitLine[currIndex+1],splitLine[currIndex+2],splitLine[currIndex+3]])
            currIndex += 4
        ls.append(ptrList)
        if (Isverb):
            f_cnt = int(splitLine[currIndex])
            currIndex += 2
            ls.append(f_cnt)
            ls_fnum_wnum = []
            for j in range(f_cnt):
                ls_fnum_wnum.append([splitLine[currIndex],splitLine[currIndex+1]])
                currIndex += 3
            ls.append(ls_fnum_wnum)
        currIndex += 1
        ls.append(" ".join(splitLine[currIndex:]))
        dataLines[i] = ls
    if (Isverb):
        df = pd.DataFrame(dataLines, columns = ['synset_offset', 'lex_filenum', 'ss_type', 'w_cnt', 'word with lex_id', 'p_cnt', 'ptr', 'f_cnt', 'frame', 'gloss'])
    else:
        df = pd.DataFrame(dataLines, columns = ['synset_offset', 'lex_filenum', 'ss_type', 'w_cnt', 'word with lex_id', 'p_cnt', 'ptr', 'gloss'])
    return df

In [6]:
def createSenseIndexDF(filePath):
    with open(filePath) as f:
        senseIndexDump = f.readlines()
    senseIndexLines = []
    for i in senseIndexDump:
        if i[0] == "." or i[0] == "'":
            continue
        senseIndexLines.append(i.split())
    df = pd.DataFrame(senseIndexLines, columns = ['sense_key', 'synset_offset', 'sense_number', 'tag_cnt'])
    return df

In [7]:
createSenseIndexDF('dict/index.sense')

,sense_key,synset_offset,sense_number,tag_cnt
0,0%1:23:00::,13742358,1,20
1,0%5:00:00:cardinal:00,02186132,1,3
2,1%1:23:00::,13742573,1,21
3,1%5:00:00:cardinal:00,02186338,1,107
4,1-dodecanol%1:27:00::,14930670,1,0
...,...,...,...,...
206919,zymosis%1:22:01::,13575433,1,0
206920,zymotic%3:01:00::,03000341,2,0
206921,zymotic%3:01:01::,03000447,1,0
206922,zymurgy%1:09:00::,06080361,1,0


In [10]:
createIndexDF('dict/index.noun')

,lemma,pos,synset_cnt,p_cnt,ptr_symbol,sense_cnt,tagsense_cnt,synset_offset
0,'hood,n,1,2,"[@, ;]",1,0,[08641944]
1,'s_gravenhage,n,1,2,"[@, #p]",1,0,[08950407]
2,0,n,1,1,[@],1,1,[13742358]
3,1,n,1,2,"[@, ~]",1,1,[13742573]
4,1-dodecanol,n,1,1,[@],1,0,[14930670]
...,...,...,...,...,...,...,...,...
117792,zymology,n,1,1,[@],1,0,[06080361]
117793,zymolysis,n,1,3,"[@, ~, +]",1,0,[13575433]
117794,zymosis,n,2,4,"[@, ~, +, ;]",2,0,"[13575433, 13575226]"
117795,zymurgy,n,1,1,[@],1,0,[06080361]
